Custom——token-autho

用自定义的 Token 认证方式访问需要 Bearer Token 的 API（如 GitHub），并打印返回结果

In [ ]:
import requests
from requests.auth import AuthBase


In [ ]:
class TokenAuth(AuthBase):
    """Implements a token authentication scheme."""

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """Attach an API token to the Authorization header."""
        r.headers["Authorization"] = f"Bearer {self.token}"
        return r


In [ ]:
if __name__ == "__main__":
    token = "<YOUR_GITHUB_PA_TOKEN>"
    response = requests.get(
        "https://api.github.com/user", auth=TokenAuth(token)
    )
    print(response.status_code)
    print(response.text)

persist-info-with-session

使用 requests 的 Session 对象来持久化认证信息，并多次发送带有自定义 Token 认证的请求

In [ ]:
import requests
from custom_token_auth import TokenAuth

In [ ]:
TOKEN = "<YOUR_GITHUB_PA_TOKEN>"

In [ ]:
with requests.Session() as session:
    session.auth = TokenAuth(TOKEN)

    first_response = session.get("https://api.github.com/user")
    second_response = session.get("https://api.github.com/user")

print(first_response.headers)
print(second_response.json())

raise error

用 requests 库处理 HTTP 请求中的异常（错误），特别是如何捕获和处理 HTTP 错误

In [ ]:
import requests
from requests.exceptions import HTTPError

In [ ]:
URLS = ["https://api.github.com", "https://api.github.com/invalid"]
for url in URLS:
    try:
        response = requests.get(url)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    else:
        print("Success!")

Retry-twice

用 requests 库为特定网站设置自动重试机制

In [ ]:
import requests  # 发送 HTTP 请求的第三方库
from requests.adapters import HTTPAdapter  # requests 的适配器，可以自定义连接行为（如重试次数）
from requests.exceptions import RetryError  # 当重试多次后仍然失败时抛出的异常

In [ ]:
github_adapter = HTTPAdapter(max_retries=2)  # 为 GitHub API 设置最大重试次数为 2，总共尝试 3 次：1 次原始 + 2 次重试）

In [ ]:
session = requests.Session()
session.mount("https://api.github.com", github_adapter) # 将适配器挂载到 GitHub API 上

In [ ]:
try:
    response = session.get("https://api.github.com/")
except RetryError as err:
    print(f"Error: {err}")
finally:
    session.close()

retry_thrice

In [ ]:
import logging
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import RetryError
from urllib3.util import Retry

In [ ]:
logging.basicConfig(level=logging.DEBUG)

In [ ]:
retries = Retry(total=3, status_forcelist=[404])

In [ ]:
github_adapter = HTTPAdapter(max_retries=retries)
session = requests.Session()
session.mount("https://api.github.com", github_adapter)
try:
    response = session.get("https://api.github.com/")
    print(response.status_code)
    response = session.get("https://api.github.com/invalid")
    print(response.status_code)
except RetryError as err:
    print(f"Error: {err}")
finally:
    session.close()
timeout_catcher
import requests
from requests.exceptions import Timeout

text-matches

使用 GitHub 搜索 API，并获取搜索结果中关于关键词匹配的详细信息

In [ ]:
response = requests.get(
    "https://api.github.com/search/repositories",
    params={"q": '"real python"'},
    headers={"Accept": "application/vnd.github.text-match+json"},
)

In [ ]:
json_response = response.json()
first_repository = json_response["items"][0]
print(first_repository["text_matches"][0]["matches"])

search_popular_repos

In [ ]:
import requests

In [ ]:
# Search GitHub's repositories for popular Python projects
response = requests.get(
    "https://api.github.com/search/repositories",
    params={"q": "language:python", "sort": "stars", "order": "desc"},
)
# q 是搜索的关键词，sort 是排序方式，order 是排序方向，星标最多的在前

In [ ]:
# Inspect some attributes of the first three repositories
json_response = response.json()  # 将响应转换为 JSON 格式
popular_repositories = json_response["items"]  # 获取搜索结果中的前三个仓库
for repo in popular_repositories[:3]:
    print(f"Name: {repo['name']}")
    print(f"Description: {repo['description']}")
    print(f"Stars: {repo['stargazers_count']}")
    print()

Timeout_catcher

In [ ]:
import requests
from requests.exceptions import Timeout

In [ ]:
try:
    response = requests.get("https://api.github.com", timeout=(3.05, 5))
except Timeout:
    print("The request timed out")
else:
    print("The request did not time out")